In [1]:
# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras.backend as K
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM
from keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from wordcloud import WordCloud
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# CREATING A DATAFRAME
df=pd.read_csv('add train data set absolute path',encoding='utf-8')
df.columns=['rating','title','review']
df=df.drop(columns=['title'])
df.insert(2,'sentiment',0)
df.loc[df['rating']>3, 'sentiment']=1
df = df.drop(columns=['rating'])


In [ ]:
# CONSIDERING 8 LAKH REVIEWS FOR THE DATAFRAME
df = df.loc[:799999]
df.describe()


In [ ]:
# CREATING A GRAPH PLOT FOR REVIEWS BASED ON SENTIMENT '0' OR'1'
sns.countplot(df['sentiment'], palette="Set1")

In [ ]:
# CALCULATING THE LENGTH of each review and align them under "TEXT_LENGTH "
df['text_length'] = df['review'].apply(len)
df[['sentiment','text_length','review']].head(15)

In [ ]:
#Plotting a graph for TEXT_LENGTH and REVIEWS
df['text_length'].hist(bins=50, color="m")

In [ ]:
# A BAR PLOT for 'TEXT_LENGTH' and 'REVIEWS' for two sentiments '0' and '1'
g = sns.FacetGrid(df,col='sentiment')
g.map(plt.hist,'text_length', color = 'g')

In [ ]:
# BOXPLOT
sns.boxplot(x='sentiment',y='text_length',data=df,palette='winter_r')

In [ ]:
# WORDCLOUD
text = df['review'].to_string()
wordcloud = WordCloud(
background_color='white',max_font_size = 24,relative_scaling=0.5,
stopwords=set(stopwords.words('english'))).generate(text)
plt.figure(figsize=(16,16))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()
X = df['review']
y = to_categorical(df['sentiment'])

#y=df['sentiment']
num_classes = df['sentiment'].nunique()
seed = 101
np.random.seed(seed)

In [ ]:
# Splitting data as train data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y,
random_state=seed)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# WORD EMBEDDING
max_features = 15000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(dict(tokenizer.word_index.items()).values())
print(list(dict(tokenizer.word_index.items()).keys())[0:200])
totalNumWords = [len(one_comment) for one_comment in X_train]
print(totalNumWords)
plt.hist(totalNumWords,bins = 30)
plt.show()
max_words = 100
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_test.shape)

In [ ]:
# BUILDING THE MODEL

batch_size = 1500
epochs =1
epochs2=2
epochs5=5
epochs10=20
max_features = 15000
embed_dim = 100
num_folds = 10
np.random.seed(seed)
K.clear_session()

model = Sequential()

model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1]))

model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(100, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',

optimizer='adam',

metrics=['accuracy'])

print(model.summary())

In [ ]:
# FITTING THE DATASET TO THE MODEL
model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
epochs=epochs10, batch_size=batch_size, verbose=1)
fig, axs = plt.subplots(1,2,figsize=(15,5))
axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
axs[0].set_title('Model Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch'+str(epochs10))
axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.
history['acc'])/10)
axs[0].legend(['train', 'val'], loc='best')
axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
axs[1].set_title('Model Loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch'+str(epochs10))
axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.
history['loss'])/10)
axs[1].legend(['train', 'val'], loc='best')
plt.show()

In [ ]:
# PREDICTION
y_pred_test = model.predict_classes(X_test, batch_size=batch_size, verbose=0)
print('Accuracy:\t{:0.1f}%'.format(accuracy_score(np.argmax(y_test,axis=1),y_pred_test)*100))
print('\n')
print(classification_report(np.argmax(y_test,axis=1), y_pred_test))
fig, ax = plt.subplots(figsize=(4, 4))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
for j in range(confmat.shape[1]):
ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()

In [ ]:
# CONFUSION MATRIX
confmat = confusion_matrix(np.argmax(y_test,axis=1), y_pred_test)